## Overview
Generative models like PaLM 2 are powerful language models used for various natural language processing (NLP) tasks. One of those is text classification, which involves assigning one or more categories to a given piece of text. Although text classification can be done using traditional NLP techniques, LLMs can perform classification by providing prompts (as opposed to domain-specific labeled data), which can accelerate the time it takes to build a text classification solution. Classification models based on LLMs can be further tuned with many examples via custom model training, but that is beyond the scope of this notebook.

In this notebook, we will explore how to do text classification using prompts with the PaLM API. Learn more about classification prompts in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/classification-prompts).

## Objective
By the end of the notebook, we should be able to use a large language model to perform various classification tasks, including:

* Zero-shot prompting text classification
* Few-shot prompting text classification
* Common tasks:
* Sentiment analysis
* Topic classification
* Spam detection
* Intent recognition
* Language identification
* Toxicity detection
* Emotion detection

In [1]:
# import libraries
import os
import vertexai
from IPython.display import Markdown, display
from google.oauth2 import service_account
from dotenv import load_dotenv
from vertexai.language_models import TextGenerationModel

In [2]:
# initiate service account (authentication)
json_path = '../llm-ai.json' # replace with your own service account
credentials = service_account.Credentials.from_service_account_file(json_path)

In [3]:
# start Vertex AI
load_dotenv()
vertexai.init(project=os.environ["PROJECT_ID"], # replace with your own project
              credentials=credentials)

In [4]:
# load the model
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### Text Classification

In the section below, you will explore zero-shot prompting, few-shot prompting, and some common types of text classification tasks.

### 1. Zero-shot prompting
Zero-shot prompting is where we do not provide examples with labels, and rely on the LLM to make the classification on its own.

In [5]:
prompt = """
Classify the following:\n
text: "I saw a furry animal in the park today with a long tail and big eyes."
label: dogs, cats
"""

label = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

# you may use print instead of display
display(
    Markdown(label)
)

The text is about a furry animal with a long tail and big eyes. It is likely to be a dog or a cat.

### 2. Few-shot prompting

With few-shot prompting, we provide examples to the PaLM model and expect it to predict classes based on the provided examples.

In [7]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

answer = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

display(
    Markdown(answer)
)

entertainment

**Other classification examples**
Explore some more common text classification prompts below, which are all based on zero-shot prompts. We can also turn some of these into few-shot prompts by providing our own custom examples of text and the associated output classes.

### 3. Topic classification

In [8]:
prompt = """
Classify a piece of text into one of several predefined topics, such as sports, politics, or entertainment. \n
text: President Biden will be visiting India in the month of March to discuss a few opportunites. \n
class:
"""

topic = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

display(
    Markdown(topic)
)

politics

### 4. Spam Detection

In [9]:
prompt = """
Given an email, classify it as spam or not spam. \n
email: hi user, \n
      you have been selected as a winner of the lotery and can win upto 1 million dollar. \n
      kindly share your bank details and we can proceed from there. \n\n

      from, \n
      US Official Lottry Depatmint
"""

detect = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

display(
    Markdown(detect)
)

spam

### 5. Intent recognition

In [10]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

intent = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

display(
    Markdown(intent)
)

making a reservation

### 6. Language identification

In [11]:
prompt = """
Given a piece of text, classify the language it is written in. \n
text: Selam nasıl gidiyor?
language:
"""

language = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

display(
    Markdown(language)
)

Turkish

### 7. Toxicity detection

In [12]:
prompt = """
Given a piece of text, classify it as toxic or non-toxic. \n
text: i love sunny days
"""

toxicity = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

display(
    Markdown(toxicity)
)

Non-toxic

### 9. Emotion detection

In [13]:
prompt = """
Given a piece of text, classify the emotion it conveys, such as happiness, or anger. \n
text: I'm still so delighted from yesterday's news
"""

emotion = generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text

display(
    Markdown(emotion)
)

happiness

### Evaluation

We can evaluate the outputs of the text classification task if the ground truth classes are available. To showcase how this works, start by creating a simple dataframe with product reviews and the ground truth sentiment.

In [17]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [18]:
review_data = {
    "review": [
        "i love this product. it does have everything i am looking for!",
        "all i can say is that you will be happy after buying this product",
        "its way too expensive and not worth the price",
        "i am feeling okay. its neither good nor too bad.",
    ],
    "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
}

review_data_df = pd.DataFrame(review_data)
review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i am looking for!,positive
1,all i can say is that you will be happy after buying this product,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


Now that we have the data with reviews and sentiments as ground truth labels, you can call the text generation model to each review row using the `apply` function. Each row will use the prompt in the `review` column to predict the sentiment using the PaLM API, and store the results in `sentiment_prediction` column.

In [19]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" and "negative". \n\n
                review: {row} \n
                sentiment:
              """
    response = generation_model.predict(prompt=prompt).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i am looking for!,positive,positive
1,all i can say is that you will be happy after buying this product,positive,positive
2,its way too expensive and not worth the price,negative,negative
3,i am feeling okay. its neither good nor too bad.,neutral,neutral


In the end, we can call the `classification_report` function from [sklearn](https://scikit-learn.org/stable/) to measure the accuracy and other classification metrics by passing ground truth sentiments `sentiment_groundtruth` and predicted sentiment `sentiment_prediction`:

In [20]:
from sklearn.metrics import classification_report

display(
        Markdown(
        classification_report(
            review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
        )
    )
)

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
    positive       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4
